## Collecting Champion Meta Data from Data Dragon (https://developer.riotgames.com/docs/lol)
Gulliver Wutz

In [1]:
import requests
import json
import time
import pytz
import datetime
import hashlib
import pandas as pd

def unix_to_cest(unix_timestamp):
    dt_utc = datetime.datetime.utcfromtimestamp(unix_timestamp)
    cest = pytz.timezone('Europe/Berlin')
    dt_cest = dt_utc.replace(tzinfo=pytz.utc).astimezone(cest)
    formatted_dt = dt_cest.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_dt

def string_to_hexadecimal(input_string):
    hash_object = hashlib.sha256()
    hash_object.update(input_string.encode())
    hex_digest = hash_object.hexdigest()
    return hex_digest

pd.set_option('display.max_colwidth', 200)

### Inspect Champion Metadata

In [2]:
# Get list of all champions
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
json_file = requests.get('https://ddragon.leagueoflegends.com/cdn/14.8.1/data/en_US/champion.json', headers=headers).text
champions = json.loads(json_file)

champion_ids = []

for champion, data in list(champions['data'].items()):
    champion_ids.append(data['id'])  
print(champion_ids)

['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'AurelionSol', 'Azir', 'Bard', 'Belveth', 'Blitzcrank', 'Brand', 'Braum', 'Briar', 'Caitlyn', 'Camille', 'Cassiopeia', 'Chogath', 'Corki', 'Darius', 'Diana', 'Draven', 'DrMundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger', 'Hwei', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'JarvanIV', 'Jax', 'Jayce', 'Jhin', 'Jinx', 'Kaisa', 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', 'Khazix', 'Kindred', 'Kled', 'KogMaw', 'KSante', 'Leblanc', 'LeeSin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'Milio', 'MissFortune', 'MonkeyKing', 'Mordekaiser', 'Morgana', 'Naafiri', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nilah', 'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Poppy', 'Pyk

In [3]:
# Get a list of all basic stat types
basic_stats = []

for stat, value in list(champions['data'][champion_ids[0]]['stats'].items()):
    basic_stats.append(stat)
print(basic_stats) 

['hp', 'hpperlevel', 'mp', 'mpperlevel', 'movespeed', 'armor', 'armorperlevel', 'spellblock', 'spellblockperlevel', 'attackrange', 'hpregen', 'hpregenperlevel', 'mpregen', 'mpregenperlevel', 'crit', 'critperlevel', 'attackdamage', 'attackdamageperlevel', 'attackspeedperlevel', 'attackspeed']


In [4]:
# Get a list of all champion classes
champion_classes = []

for champ_id in champion_ids:
    for cclass in champions['data'][champ_id]['tags']:
        champion_classes.append(cclass)
champion_classes = sorted(list(set(champion_classes)))
print(champion_classes)

['Assassin', 'Fighter', 'Mage', 'Marksman', 'Support', 'Tank']


In [5]:
# Get the number of skins available for a champion
champ_json_file = requests.get('https://ddragon.leagueoflegends.com/cdn/14.4.1/data/en_US/champion/Aatrox.json', headers=headers).text
champion_details = json.loads(champ_json_file)

print('Number of skins: ')
print(len(champion_details['data']['Aatrox']['skins']))

Number of skins: 
12


In [6]:
# Get basic stats of spells per level

print('Spell Cooldown:')
print(champion_details['data']['Aatrox']['spells'][0]['cooldown'])
print('')

print('Spell Cost:')
print(champion_details['data']['Aatrox']['spells'][0]['cost'])
print('')

print('Spell Range:')
print(champion_details['data']['Aatrox']['spells'][0]['range'])
print('')

Spell Cooldown:
[14, 12, 10, 8, 6]

Spell Cost:
[0, 0, 0, 0, 0]

Spell Range:
[25000, 25000, 25000, 25000, 25000]



### Collect Champion Metadata

In [7]:
# Create columns for spell stats
spell_columns = []
spells = ['Q', 'W', 'E', 'R']

for spell in spells:
    colname1 = spell + ' Cooldown'
    spell_columns.append(colname1)
    colname2 = spell + ' Cost'
    spell_columns.append(colname2)
    colname3 = spell + ' Range'
    spell_columns.append(colname3)

columns = ['Name', 'Patch']
columns.extend(basic_stats)
columns.extend(spell_columns)
columns.extend(champion_classes)
columns.extend(['Skins', 'Collected CEST', 'Sample ID'])


# Patch Criteria
patch_criteria = {'14.8': '14.8.1', '14.7': '14.7.1', '14.6': '14.6.1', '14.5': '14.5.1', '14.4': '14.4.1'}

champion_meta_df = pd.DataFrame(columns=columns)

for champion_id in champion_ids:
    for patch_name, patch in patch_criteria.items():

        new_data = []
        current_timestamp = int(time.time())
        temp_link = 'https://ddragon.leagueoflegends.com/cdn/' + patch + '/data/en_US/champion/' + champion_id + '.json'
        champ_json_file = requests.get(temp_link, headers=headers).text
        champion_details = json.loads(champ_json_file)

        print('Collecting ' + temp_link)
        print('Configuration: ' + champion_id + ' - ' + patch_name)
        print('')
        

        # Name
        new_data.append(champion_id)

        # Patch
        new_data.append(patch_name)
        
        # Basic Stats
        for stat, value in list(champion_details['data'][champion_id]['stats'].items()):
            new_data.append(value)

        # Spell Stats
        for i in range(4):
            for stat_type in ['cooldown', 'cost', 'range']:
                new_data.append(str(champion_details['data'][champion_id]['spells'][i][stat_type]).replace('[', '').replace(']', '').replace(', ', '/'))

        # Champion Classes
        tags = champions['data'][champion_id]['tags']
        for cclass in champion_classes:
            if cclass in tags:
                new_data.append(1)
            else:
                new_data.append(0)

        # Number of Skins
        new_data.append(len(champion_details['data'][champion_id]['skins']))

        # Timestamp
        new_data.append(unix_to_cest(current_timestamp))

        # Sample ID
        new_data.append(string_to_hexadecimal(champion_id + patch_name))
        

        new_row = pd.DataFrame([new_data], columns=champion_meta_df.columns)
        champion_meta_df = pd.concat([champion_meta_df, new_row], ignore_index=True)
        champion_meta_df.to_csv('champion_stats_ddragon.csv', index=False)

        time.sleep(1)



Configuration: Aatrox - 14.8

Configuration: Aatrox - 14.7

Configuration: Aatrox - 14.6

Configuration: Aatrox - 14.5

Configuration: Aatrox - 14.4

Configuration: Ahri - 14.8

Configuration: Ahri - 14.7

Configuration: Ahri - 14.6

Configuration: Ahri - 14.5

Configuration: Ahri - 14.4

Configuration: Akali - 14.8

Configuration: Akali - 14.7

Configuration: Akali - 14.6

Configuration: Akali - 14.5

Configuration: Akali - 14.4

Configuration: Akshan - 14.8

Configuration: Akshan - 14.7

Configuration: Akshan - 14.6

Configuration: Akshan - 14.5

Configuration: Akshan - 14.4

Configuration: Alistar - 14.8

Configuration: Alistar - 14.7

Configuration: Alistar - 14.6

Configuration: Alistar - 14.5

Configuration: Alistar - 14.4

Configuration: Amumu - 14.8

Configuration: Amumu - 14.7

Configuration: Amumu - 14.6

Configuration: Amumu - 14.5

Configuration: Amumu - 14.4

Configuration: Anivia - 14.8

Configuration: Anivia - 14.7

Configuration: Anivia - 14.6

Configuration: Anivia - 1

### Inspect Dataset

In [143]:
# Check if Sample ID hashes are ideed uniquely generated
champion_meta_df = pd.read_csv('champion_stats_ddragon.csv')
print("Sample ID column contains duplicates: " + str(champion_meta_df['Sample ID'].duplicated().any()))
print("Dataset contains NaNs: " + str(champion_meta_df.isna().any().any()))

Sample ID column contains duplicates: False
Dataset contains NaNs: False


In [151]:
# Print out all unique values of a selected column
champion_meta_df['Patch'].unique().tolist()

[14.8, 14.7, 14.6, 14.5, 14.4]

In [153]:
# Inspect Dataset Types
champion_meta_df.dtypes

Name                     object
Patch                   float64
hp                        int64
hpperlevel                int64
mp                        int64
mpperlevel              float64
movespeed                 int64
armor                     int64
armorperlevel           float64
spellblock                int64
spellblockperlevel      float64
attackrange               int64
hpregen                 float64
hpregenperlevel         float64
mpregen                 float64
mpregenperlevel         float64
crit                      int64
critperlevel              int64
attackdamage              int64
attackdamageperlevel    float64
attackspeedperlevel     float64
attackspeed             float64
Q Cooldown               object
Q Cost                   object
Q Range                  object
W Cooldown               object
W Cost                   object
W Range                  object
E Cooldown               object
E Cost                   object
E Range                  object
R Cooldo

In [154]:
# Inspect Dataset
champion_meta_df

,Name,Patch,hp,hpperlevel,mp,mpperlevel,movespeed,armor,armorperlevel,spellblock,...,R Range,Assassin,Fighter,Mage,Marksman,Support,Tank,Skins,Collected CEST,Sample ID
0,Aatrox,14.8,650,114,0,0.0,345,38,4.45,32,...,25000/25000/25000,0,1,0,0,0,1,12,2024-04-26 13:12:43,b4d8e72ce2657a5bc9d3816e19ee1c99381c4827dc3a497a6e4841555aad0b3b
1,Aatrox,14.7,650,114,0,0.0,345,38,4.45,32,...,25000/25000/25000,0,1,0,0,0,1,12,2024-04-26 13:12:44,60feb780601dc8c5e3a2f68b38bb77ce24bec0c754e5f2d43ebec2c774cd2208
2,Aatrox,14.6,650,114,0,0.0,345,38,4.45,32,...,25000/25000/25000,0,1,0,0,0,1,12,2024-04-26 13:12:45,0dd02f4554590d2a870eade186b933ad9f4b485b3c46f19329ddc6168595dc6e
3,Aatrox,14.5,650,114,0,0.0,345,38,4.45,32,...,25000/25000/25000,0,1,0,0,0,1,12,2024-04-26 13:12:46,b68b7b864ad7e63870f8a68aeddec448642cb32c07903d2118c0362a3ce66cd2
4,Aatrox,14.4,650,114,0,0.0,345,38,4.45,32,...,25000/25000/25000,0,1,0,0,0,1,12,2024-04-26 13:12:47,918d6f4ace8587b9962cdbe22f962c72ecbfffdbd4166faa3564bf670c49fb8e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,Zyra,14.8,574,93,418,25.0,340,29,4.20,30,...,700/700/700,0,0,1,0,1,0,12,2024-04-26 13:32:50,535760de3707ec6f41edadcd03e40e431996fa5062ab9ed75f8f408824e31129
831,Zyra,14.7,574,93,418,25.0,340,29,4.20,30,...,700/700/700,0,0,1,0,1,0,11,2024-04-26 13:32:51,ef2977ccb7789c8fc3f82cd2c1b4b3410abc89e26b8bceb3c383badec34d7e22
832,Zyra,14.6,574,93,418,25.0,340,29,4.20,30,...,700/700/700,0,0,1,0,1,0,11,2024-04-26 13:32:52,8739928ae492552050d231f5edbe02ca40b64e7f5ed7abddcf53b72fb6dfbd1a
833,Zyra,14.5,574,93,418,25.0,340,29,4.20,30,...,700/700/700,0,0,1,0,1,0,11,2024-04-26 13:32:54,875b0e27cb2b2752e37a52791aa35b11cba5410102cced029528c115396bfc0e
